In [47]:
import numpy as np
import pandas as pd
from collections import defaultdict
import os
from dataDictionary import makeDataDic

In [48]:
fname = "updateAds2.xlsx"
dataset = pd.read_excel(fname)
dataset = dataset.loc[:, ~dataset.columns.str.contains('^Unnamed')]
X = dataset.iloc[:, :].values

In [49]:
count = len(X[0, :])
if count == 5:
    isRequst = True
else:
    isRequst = False

In [50]:
mobj = fname[0:fname.index(".")-1]
true_name = mobj[0].capitalize() + mobj[1:]
if isRequst:
  main_name = mobj  
else:
  main_name = mobj + "Response"

In [51]:
c = mobj[0].capitalize()
mobj = c + mobj[1:]
mobj
if isRequst:
    mobj += "Type"
else:
    mobj += "ResponseType"

In [52]:
mobj

'UpdateAdsResponseType'

In [53]:
result = f"""
    {mobj}:
      type: object
      properties:
"""

In [54]:
print(result)


    UpdateAdsResponseType:
      type: object
      properties:



In [55]:
result += f"""        {main_name}:
          type: object
          properties: 
"""

In [56]:
print(result)


    UpdateAdsResponseType:
      type: object
      properties:
        updateAdsResponse:
          type: object
          properties: 



In [57]:
def camel_case(string):
  c = string[0].lower()
  string = c + string[1:]
  return string

In [58]:
required = defaultdict(lambda: [])
if isRequst:
  for row in X:
    tag = row[0].split('>')
    tag = [el.strip() for el in tag]
    tag = [el.lower() for el in tag]
    if row[-2].lower() == 'y':
      required[tag[0]].append(tag[-1])

In [59]:
def spaces(string):
  return ''.join(['  ' for s in range(string.count('>'))])

In [60]:
def extractNandM(string:str):
  n = ""
  m = ""
  nfound = True
  ffound = False
  for el in string:
    if el.isdigit():
      ffound = True
      if nfound:
        n += el
      else:
        m += el
    else:
      if ffound:
        nfound = False
  return [n, m]

In [61]:
def makeTSpaces(num, mCount=0, mmCount=0):
  if mmCount > 1 and mCount != 0:
    num += mmCount 
  return ''.join(['  ']*num)

In [62]:
def checkDefault(default):
  if type(default) != float and default:
    return f"""default: {default}"""
  else:
    return ""

In [63]:
def ismatchany(pre, curr):
    i = 0
    mCount = 0
    tCount = len(pre) if len(pre) > len(curr) else len(curr)
    while i < len(pre) and i < len(curr):
      if pre[i] == curr[i]:
        mCount += 1
      else:
        break
      i += 1
    return [True if mCount > 0 else False, mCount, tCount - mCount]

In [64]:
def camel_case(s):
  return ''.join([s[0].lower(), s[1:]])

In [65]:
def initalization(lst):
  obj = """"""
  robj = """"""
  rflag = False
  tag = lst[0]
  data = lst[1].replace(' ', '')
  if isRequst:
    default = lst[2]
    req = lst[3]
  else:
    default = None
    req = None
  des = lst[-1]
  tag = tag.split('>')
  tag = [el.replace(' ', '') for el in tag]
  tag = [camel_case(el) for el in tag]
  tLength = len(tag)
  return tag, data,default, req, des, obj, robj, rflag, tLength, 0

In [66]:
def findenum(des):
  des = des.split()
  s = set()
  for word in des:
    word = ''.join(c for c in word if c.isalpha())
    if len(word) == 1 and word.isupper():
      s.add(word)
  return s

In [67]:
def makeObject(lst, pre, ts=0, o=0, dSpaces=2):
  tag, data, default, req, des, obj, robj, rflag, tLength, i = initalization(lst)
  elist = None

  n, m = extractNandM(data)

  if n == '1' and m == '':
    elist = findenum(des)

  ismatch, mCount, mmCount = ismatchany(pre, tag)
  
  if ismatch:
    tag = tag[mCount:]
    if len(pre) >= tLength:
      o -= (mmCount - 1)
      ts -= mmCount
    else:
      o = mCount
      ts = mCount
  else:
    o = 0
    ts = 0
    key = tag[0].lower()
    if len(required[key]) != 0:
      robj += f"""{makeTSpaces(dSpaces+1)}required:\n"""
      lst = required[key]
      rflag = True
      for el in lst:
        robj += f"""{makeTSpaces(dSpaces+2)}- {el}\n"""
  
  length = len(tag)

  while i < length-1:
    obj += f"""{makeTSpaces(o+ts+dSpaces)}{tag[i].lower() if len(tag[i]) < 4 else tag[i]}:\n"""
    ts += 1
    obj += f"""{makeTSpaces(o+ts+dSpaces)}type: object\n"""
    if robj != "" and rflag:
      obj += robj
      rflag = False
    obj += f"""{makeTSpaces(o+ts+dSpaces)}properties:\n"""
    o += 1
    i += 1

  obj += f"""{makeTSpaces(o+ts+dSpaces)}{tag[i].lower() if len(tag[i]) < 4 else tag[i]}:\n"""
  ts += 1
  obj += f"""{makeTSpaces(o+ts+dSpaces)}{makeDataDic(isRequst, n, m,makeTSpaces(o+ts+dSpaces))[data]}\n"""
  default = checkDefault(default)
  if default:
    obj += f"""{makeTSpaces(o+ts+dSpaces)}{default}\n"""
  if elist and isRequst:
    obj += f"""{makeTSpaces(o+ts+dSpaces)}enum:\n"""
    for el in elist:
      obj += f"""{makeTSpaces(o+ts+dSpaces+1)}- {el}\n"""
  obj += f"""{makeTSpaces(o+ts+dSpaces)}description: >\n"""
  des = des.split('\n')
  for line in des:
    obj += f"""{makeTSpaces(o+ts+1+dSpaces)}{line}\n"""
  if req and req.lower() == 'c':
    obj += f"""{makeTSpaces(o+ts+1+dSpaces)}This field is conditional.\n"""
  return obj, ts, o


In [68]:
pre = []
if isRequst:
  dSpaces = 2
else:
  dSpaces = 6
bigObj = """"""
ts = 0
o = 0
aObj = """"""
caObj = """"""
cObj = """"""
aFlag = True
caFlag = True
cFlag = True
for row in X:
  curr = row
  if isRequst:
    tag = curr[0].split('>')
    tag = tag[0].strip().lower()
    if tag == 'acquirer':
      if aFlag:
        result += f"""{makeTSpaces(6)}acquirer:\n"""
        result += f"""{makeTSpaces(7)}$ref: '#/components/schemas/AcquirerType_{true_name}'\n"""
        aFlag = False
      obj, ts, o = makeObject(curr, pre, ts, o, dSpaces)
      aObj += obj
    elif tag == 'cardacceptor':
      if caFlag:
        result += f"""{makeTSpaces(6)}cardAcceptor:\n"""
        result += f"""{makeTSpaces(7)}$ref: '#/components/schemas/CardAcceptorType_{true_name}'\n"""
        caFlag = False
      obj, ts, o = makeObject(curr, pre, ts, o, dSpaces)
      caObj += obj
    elif tag == 'card':
      if cFlag:
        result += f"""{makeTSpaces(6)}card:\n"""
        result += f"""{makeTSpaces(7)}$ref: '#/components/schemas/CardType_{true_name}'\n"""
        cFlag = False
      obj, ts, o = makeObject(curr, pre, ts, o, dSpaces)
      cObj += obj
    else:
      obj, ts, o = makeObject(curr, pre, ts, o, 6)
      result += obj
  else:
    obj, ts, o = makeObject(curr, pre, ts, o, dSpaces)
    result += obj
  pre = curr[0].replace(" ", "").split('>')
  pre = [camel_case(el) for el in pre]

if isRequst:
  result += aObj
  result += caObj
  result += cObj

result = os.linesep.join([s for s in result.splitlines() if s])

In [69]:
print(result)

    UpdateAdsResponseType:
      type: object
      properties:
        updateAdsResponse:
          type: object
          properties: 
            responseCode:
              type: string
              format: AN4
              description: >
                Response code sent from the MCP server See Appendix Response Codes for details.
            responseDesc:
              type: string
              format: Strg
              description: >
                Response code description or the response message
            aDSGroups:
              type: object
              properties:
                aDSGroup:
                  type: object
                  properties:
                    ads:
                      type: object
                      properties:
                        type:
                          type: string
                          format: Strg5
                          description: >
                            Group Types
                            Please se